### Set Up

In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
from torch import optim 

from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir("/content/gdrive/MyDrive/CS640/Project/Model")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Dataset

In [2]:
class Dataset(Dataset):

    def __init__(self, file_path):
        self.df = pd.read_csv(file_path)

    def __getitem__(self, idx):
        X = np.array(self.df.iloc[idx, :-1]).astype(np.float32)
        y = self.df.iloc[idx, -1]

        return X, y

    def __len__(self):
        return len(self.df)

In [3]:
demoDataset = Dataset("./demoData.csv")

train_size = 16 ## __len__
test_size = len(demoDataset) - train_size

full_train, test = random_split(demoDataset, [train_size, test_size])

train_size = 12 ## __len__
valid_size = len(full_train) - train_size

train, valid = random_split(full_train, [train_size, valid_size])

print(f'Full: {len(demoDataset)}')
print(f'Train: {len(train)}')
print(f'Valid: {len(valid)}')
print(f'Test: {len(test)}')

Full: 20
Train: 12
Valid: 4
Test: 4


### DataLoader

In [4]:
bs = 4

train_loader = DataLoader(train, batch_size=bs, shuffle=True, num_workers=2, pin_memory=True)
valid_loader = DataLoader(valid, batch_size=bs, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test, batch_size=bs, shuffle=False, num_workers=2, pin_memory=True)

### Nerual Network

In [5]:
d_in, d_out = 4, 2

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(d_in, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, d_out)

        self.bn1 = nn.BatchNorm1d(d_in)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(64)

    def forward(self, x):
        x = self.bn1(x)
        x = F.relu(self.fc1(x))
        x = self.bn2(x)
        x = F.relu(self.fc2(x))
        x = self.bn3(x)
        x = self.fc3(x)
        return x

In [6]:
def train(net, train_loader, valid_loader, num_epoch):
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(net.parameters(), lr=0.001)

  for epoch in range(num_epoch):
      running_loss = 0
      for inputs, labels in train_loader:
          inputs, labels = inputs.to(device), labels.to(device)
          net.train()
          outputs = net(inputs) # forward()
          loss = criterion(outputs, labels)
          
          optimizer.zero_grad() # clear the previous gradients 
          loss.backward()
          optimizer.step() # w = w - eta*w.grad

          running_loss += loss.item()

      # validation
      with torch.no_grad():
          val_running_loss = 0
          for inputs, labels in valid_loader:
              inputs, labels = inputs.to(device), labels.to(device)
              net.eval()
              outputs = net(inputs) # forward()
              loss = criterion(outputs, labels)
              val_running_loss += loss.item()
      
      print(f'Epoch {epoch} Train {running_loss/len(train_loader)} Valid {val_running_loss/len(valid_loader)}')


In [7]:
def eval(net, test_loader):
    net.eval()
    y_pred = []
    y_test = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs, 1)
            y_pred.extend(predicted.cpu().detach().numpy())
            y_test.extend(labels.cpu().detach().numpy())
        print("inputs:", inputs.cpu())
        print("labels", labels.cpu())
        print("results", predicted.cpu())

In [8]:
net = Net().to(device)
train(net, train_loader, valid_loader, 10)
eval(net, test_loader)

Epoch 0 Train 0.8438981771469116 Valid 0.679290771484375
Epoch 1 Train 0.3694289028644562 Valid 0.6608772873878479
Epoch 2 Train 0.33733059962590534 Valid 0.6274055242538452
Epoch 3 Train 0.2952880660692851 Valid 0.5695760250091553
Epoch 4 Train 0.23285994430383047 Valid 0.5073519945144653
Epoch 5 Train 0.21837312479813895 Valid 0.44018813967704773
Epoch 6 Train 0.5378365268309911 Valid 0.39887598156929016
Epoch 7 Train 0.572500690817833 Valid 0.34240132570266724
Epoch 8 Train 0.497586190700531 Valid 0.31825900077819824
Epoch 9 Train 0.5025555789470673 Valid 0.2946831285953522
inputs: tensor([[19., 19., 19., 19.],
        [39., 39., 39., 39.],
        [15., 15., 15., 15.],
        [63., 63., 63., 63.]])
labels tensor([0, 0, 0, 1])
results tensor([0, 0, 0, 1])
